# Post Processing Notebook for Comparisons between E+, Measured Data, and CFD..
* 02/12/23

In [81]:
from ladybug.sql import SQLiteResult
from ladybug.analysisperiod import AnalysisPeriod as ap
import numpy as np
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

In [3]:
dir = "../rosseRoomModel/rr_SurfaceConst021223/run/eplusout.sql"

In [5]:
sql_path = os.path.join(dir)
sqld = SQLiteResult(sql_path)

In [6]:
sqld.data_collections_by_output_name("Electricity:Facility")[-1].total

165.18896996250774

In [7]:
sqld.available_outputs

('Site Outdoor Air Drybulb Temperature',
 'Site Direct Solar Radiation Rate per Area',
 'Zone Air Relative Humidity',
 'Site Outdoor Air Drybulb Temperature',
 'Site Diffuse Solar Radiation Rate per Area',
 'People Convective Heating Energy',
 'People Occupant Count',
 'Zone Air Temperature',
 'Electricity:Facility',
 'People Air Relative Humidity',
 'Surface Outside Face Temperature',
 'Electricity:Facility',
 'People Air Temperature',
 'People Total Heating Energy',
 'Surface Inside Face Temperature',
 'Site Outdoor Air Wetbulb Temperature',
 'People Total Heating Rate')

## Zone vs Outdoor Air Analysis

In [40]:
# July 20 noon (actually like 7am, but ok) to July 23 noon 
exp01_ap = ap(st_month=7, st_day=20, end_month=7, end_day=23,  is_leap_year=True)
# exp01_ap.datetimes

In [44]:
zone_air = sqld.data_collections_by_output_name("Zone Air Temperature")[0]
zone_air_exp01 = zone_air.filter_by_analysis_period(exp01_ap) #.datetimes

In [51]:
amb_air = sqld.data_collections_by_output_name("Site Outdoor Air Drybulb Temperature")[0]
amb_air_exp01 = amb_air.filter_by_analysis_period(exp01_ap)

In [53]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=zone_air_exp01.datetimes,
    y=zone_air_exp01.values, 
    mode='lines+markers',
    name="Zone Air Temperature"
))
fig.add_trace(go.Scatter(
    x=amb_air_exp01.datetimes,
    y=amb_air_exp01.values, 
    mode='lines+markers',
    name="Site Outdoor Air Temperature"
))

fig.update_layout(title='rr_SurfaceConst021223',
                   xaxis_title='Dates',
                   yaxis_title='Temperature (ºC)')


## Surface Analysis

In [78]:
in_surf = sqld.data_collections_by_output_name("Surface Inside Face Temperature")
out_surf = sqld.data_collections_by_output_name("Surface Outside Face Temperature")

In [64]:
surf_names = []
for surf in out_surf:
    surf_names.append(surf.header.to_dict()["metadata"]["Surface"])
surf_names

['SURFACE 2',
 'SURFACE 3',
 'SURFACE 4',
 'SURFACE 5 OUTDOORS',
 'SURFACE 1',
 'SURFACE 6',
 'SUB SURFACE 1']

In [89]:
fig = make_subplots(rows=1, cols=7, shared_yaxes=True, subplot_titles=surf_names
)

for ix, (outs, ins) in enumerate(zip(out_surf, in_surf)):
    showlegendbool = True if ix == 0 else False

    fig.add_trace(go.Scatter(
    x=outs.filter_by_analysis_period(exp01_ap).datetimes,
    y=outs.filter_by_analysis_period(exp01_ap).values,
    name="Outdoor Surface",
    showlegend=showlegendbool,
    marker_color="blue"
    ), row=1, col=ix+1)

    fig.add_trace(go.Scatter(
    x=ins.filter_by_analysis_period(exp01_ap).datetimes,
    y=ins.filter_by_analysis_period(exp01_ap).values,
    name="Indoor Surface",
    showlegend=showlegendbool,
    marker_color="orange",
    ), row=1, col=ix+1)
fig.show()